# Python Foundations:  Generators
New Python syntax / concepts - generator functions and expressions

Foundations notebook available on Github from the powderflask/cap-comp215 repository.
As usual, the first code block just imports the modules we will use.

In [23]:
import math
from pprint import pprint


## Generator expressions
A `generator` is a series of values that may only be accessed in sequence, from the start.

Once your algorithm "consumes" a value from the generator, it is gone and cannot be retrieved again.
Thus, unlike a `list`, a `generator` can have infinite length - we will see some examples of this later.

These properties allow generators to be very efficient - they are generally designed to perform any computation required for each value "just in time", and thus consume almost no memory since the next data value doesn't actually exist until it is "consumed".  That's confusing!  Let's look at an example...

A `generator expression` looks like a list comprehension, but uses parentheses (round brackets):

In [24]:
# 3-tuples of adjacent integers...
neighbours = ((i-1, i, i+1) for i in range(1, 10))
print(neighbours)
# we can pull one item at a time out of a generator using the built-in next() function
print(next(neighbours))
print(next(neighbours))
# we can turn a generator into a list to examine the rest of its elements (though this somewhat defeats the purpose!)
print(list(neighbours))
# Notice: once the "stream" has flowed past, it is empty - there is no way to "replenish" it.
print(list(neighbours))

<generator object <genexpr> at 0x14f4ada50>
(0, 1, 2)
(1, 2, 3)
[(2, 3, 4), (3, 4, 5), (4, 5, 6), (5, 6, 7), (6, 7, 8), (7, 8, 9), (8, 9, 10)]
[]


## Generator Functions
Generator expressions are very powerful when used effectively, but are generally limited to simple map and filter operations.  If your generator needs a more complex algorithm, or you want it to be more reusable, you can write it as a function.

Here is the same generator written as a function:

In [25]:
def neighbourhoods(rng:range) -> tuple:
    """ Generate 3-tuple neighbourhoods for each value in given range object """
    for i in rng:
        yield (i-1, i, i+1)
        
n = neighbourhoods(range(1, 10))
print(n)
print(next(n))
print(next(n))
print(list(n))
print(list(n))

<generator object neighbourhoods at 0x14f4adac0>
(0, 1, 2)
(1, 2, 3)
[(2, 3, 4), (3, 4, 5), (4, 5, 6), (5, 6, 7), (6, 7, 8), (7, 8, 9), (8, 9, 10)]
[]


## Infinite Generator
It is tempting to think of a generator as being similar to a list or tuple.  That's a bad model and will cause confusion and bugs.  A generator is best thought of as a "stream of values".  Each time you get the `next` value, you "consume" it, removing that value from the stream.   That's not how a list works at all!

It is easiest to see the differences with an "infinite" generator - no such list can possibly exist, since the computer has a finite amount of memory in which to store the list.  But remember, generators a not stored in memory, their values are computed "just in time"!

Some examples of "infinite generators" would be the digits of `pi`, or all prime numbers...

In [26]:
# Example: a generator of prime numbers
# Note: These naive algorithms are to illustrate generators only.  There are more efficient ways to generate prime numbers! 

def is_prime(n):
    """ Return True iff integer n is a prime number """
    # Note
    assert type(n) is int and n >= 2
    for d in range (2, round(math.sqrt(n)) + 1):
        if n % d == 0: 
            return False
    return True

assert is_prime(11)
assert is_prime(37)
assert not is_prime(9)

In [27]:
def primes():
    """ An infinite stream of prime numbers.  Warning: do not try to make a list out of this!! """
    n = 2
    while True:
        while not is_prime(n):
            n+=1
        yield n
        n+=1

p = primes()
N = 200
pprint(f"First {N} prime numbers: {[next(p) for _ in range(N)]}")
# A generator function can be re-used by calling it again...
pprint(f"Sum of first {N} prime numbers: {sum(p for p,_ in zip(primes(), range(N)))}")

('First 200 prime numbers: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, '
 '43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, '
 '127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, '
 '199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, '
 '283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, '
 '383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, '
 '467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, '
 '577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, '
 '661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, '
 '769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, '
 '877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, '
 '983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, '
 '1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117,

### Generators everywhere!!!
Once you learn to see them, you'll spot generators everywhere (at least in well-written code).

In the last line of code from the example above, there are **4** generators!

    sum(p for p,_ in zip(primes(), range(N)))

1. `(p for p,_ in ....)`  a simple generator expression
2. `zip(...)`  zip is a generator function that returns n-tuples from its input sequences
3. `range(...)`  range is also a generator function that returns integers in a series
4. `primes()`  of course, the generator function we just wrote.